# MAGICAL CHATBOT _ LUMOS

In [65]:
import csv
import difflib
import random
import string
from tkinter import *
from tkinter import scrolledtext
from tkinter.font import Font


In [66]:
# --- Step 1: Load Data from CSV Files ---
def load_csv(filename, encoding='utf-8'):
    """Load data from a CSV file with a specified encoding and return as a list of dictionaries."""
    rows = []
    try:
        with open(filename, 'r', encoding=encoding) as file:
            reader = csv.DictReader(file)
            for row in reader:
                if 'Command' in row and 'Intent' in row:  # Ensure required fields exist
                    rows.append(row)
                elif 'Riddle' in row and 'Answer' in row:  # Special case for riddles
                    rows.append({'Riddle': row['Riddle'], 'Answer': row['Answer']})
                elif 'Input Phrase' in row and 'Response' in row:  # Special case for responses
                    rows.append({'Input Phrase': row['Input Phrase'], 'Response': row['Response']})
                elif 'Question' in row and 'Answer' in row:  # Special case for trivia questions
                    rows.append({'Question': row['Question'], 'Answer': row['Answer']})
                else:
                    print(f"Warning: Missing expected keys in row: {row}")
    except UnicodeDecodeError:
        print(f"Error: Unable to decode the file '{filename}' using {encoding}. Trying with 'ISO-8859-1'.")
        return load_csv(filename, encoding='ISO-8859-1')
    return rows

In [67]:
# Load datasets
spell_commands = load_csv('spell_commands.csv')
trivia_questions = load_csv('trivia_questions.csv')
magical_responses = load_csv('magical_responses.csv')
riddles = load_csv('riddles.csv')

In [68]:
# --- Step 2: Fuzzy Matching for Commands ---
def normalize_input(user_input):
    """Normalize user input by converting to lowercase, and removing unnecessary punctuation."""
    user_input = user_input.strip().lower().translate(str.maketrans('', '', string.punctuation.replace("'", "").replace("-", "")))
    return user_input

In [69]:
def get_intent(user_input):
    """Match user input to the closest intent using fuzzy matching."""
    commands = [cmd['Command'].strip().lower() for cmd in spell_commands]  # Normalize commands
    user_input_normalized = normalize_input(user_input)  # Normalize user input
    closest_match = difflib.get_close_matches(user_input_normalized, commands, n=1, cutoff=0.7)  # Lower cutoff for more leniency
    
    if closest_match:
        for cmd in spell_commands:
            if cmd['Command'].strip().lower() == closest_match[0]:  # Normalize comparison
                return cmd['Intent']
    return None

In [70]:
# --- Step 3: Command Handling ---
def handle_trivia():
    """Ask a random trivia question."""
    question = random.choice(trivia_questions)
    return question['Question'], question['Answer']

def handle_riddles():
    """Ask a random riddle."""
    riddle = random.choice(riddles)
    return riddle['Riddle'], riddle['Answer']

def handle_knowledge():
    """Provide general knowledge."""
    return "Here's a magical fact: The only way to move between different magical realms is by casting a powerful portkey spell."

def handle_history():
    """Provide a historical fact."""
    return "Did you know that the first magical duel ever recorded happened in 1547 between two great wizards?"

def handle_explanation():
    """Provide an explanation for a topic."""
    return "Explanation: The spell 'Alohomora' unlocks doors, but it works only on doors that are magically locked."

def handle_question():
    """Provide a magical quiz question."""
    return "What is the most powerful wand in the wizarding world?"

def handle_movie():
    """Fetch a random magical movie."""
    movies = ["Harry Potter and the Philosopher's Stone", "Fantastic Beasts and Where to Find Them", "The Sorcerer's Apprentice"]
    return random.choice(movies)

def handle_cheer():
    """Send a cheerful message."""
    return "You are magical, keep up the good work! ✨"

def handle_details():
    """Fetch detailed information."""
    return "The Wizarding World is vast and filled with magic. From wands to spells, there’s so much to explore!"

In [71]:
# --- Step 4: Sorting Hat Function ---
class SortingHat:
    def __init__(self):
        self.questions = [
            ("What do you value most?", ["Bravery", "Ambition", "Hard work", "Intelligence"]),
            ("Which would you prefer to be known for?", ["Courage", "Power", "Loyalty", "Wit"]),
            ("Pick your favorite activity:", ["Adventure", "Leadership", "Helping others", "Studying"]),
        ]
        self.current_question_index = 0
        self.answers = []

    def get_next_question(self):
        if self.current_question_index < len(self.questions):
            question, options = self.questions[self.current_question_index]
            return question, options
        else:
            return None, None

    def submit_answer(self, answer):
        # Normalize the answer before storing it
        normalized_answer = normalize_input(answer)
        self.answers.append(normalized_answer)
        self.current_question_index += 1

    def get_house(self):
        house_scores = {
            'Gryffindor': self.answers.count('bravery') + self.answers.count('courage') + self.answers.count('adventure'),
            'Slytherin': self.answers.count('ambition') + self.answers.count('power') + self.answers.count('leadership'),
            'Hufflepuff': self.answers.count('hard work') + self.answers.count('loyalty') + self.answers.count('helping others'),
            'Ravenclaw': self.answers.count('intelligence') + self.answers.count('wit') + self.answers.count('studying')
        }
        
        # Debugging: Print the scores for each house
        print("House Scores:", house_scores)

        sorted_house = max(house_scores, key=house_scores.get)
        return f"Congratulations! You have been sorted into {sorted_house}!"

sorting_hat = SortingHat()

In [72]:
# --- Step 5: Generate Response ---
current_question = None
current_answer = None
current_mode = None
current_riddle = None
current_riddle_answer = None

In [73]:
def get_response(user_input):
    """Generate a response based on user intent."""
    global current_question, current_answer, current_mode, current_riddle, current_riddle_answer
    user_input_normalized = normalize_input(user_input)  # Normalize user input

    # Check for greetings first
    if user_input_normalized in ['hi', 'hello', 'hey', 'hru', 'howdy']:
        return "Ah, greetings young wizard! How can I assist you on this magical journey?"

    # Check for input phrases in magical_responses
    for phrase in magical_responses:
        if user_input_normalized in phrase['Input Phrase'].lower():
            return phrase['Response']

    # Exit condition for trivia or riddle mode
    if user_input_normalized in ['bye', 'exit', 'quit']:
        current_mode = None
        return "Farewell, brave spellcaster! May the winds guide your path."

    # Sorting Hat mode
    if user_input_normalized == 'sort me':
        current_mode = 'sorting_hat'
        question, options = sorting_hat.get_next_question()
        if question:
            return f"{question} ({', '.join(options)})"
        else:
            return "You've already been sorted!"

    # If in sorting hat mode, evaluate the answer
    if current_mode == 'sorting_hat':
        sorting_hat.submit_answer(user_input_normalized)
        question, options = sorting_hat.get_next_question()
        if question:
            return f"{question} ({', '.join(options)})"
        else:
            result = sorting_hat.get_house()
            current_mode = None  # Reset mode after sorting
            return result

    # If in trivia mode, evaluate the answer
    if current_mode == 'trivia':
        normalized_answer = normalize_input(current_answer)
        normalized_user_input = normalize_input(user_input)
        if difflib.SequenceMatcher(None, normalized_user_input, normalized_answer).ratio() > 0.85:
            current_mode = None
            return "Correct! You've proven your magical knowledge, wizard! Type another spell to continue."
        else:
            current_mode = None
            return f"Oops, that's not quite right. The correct answer was '{current_answer}'. Would you like to try another trivia question or cast a spell?"

    # If in riddle mode, evaluate the answer
    if current_mode == 'riddle':
        normalized_riddle_answer = normalize_input(current_riddle_answer)
        normalized_user_input = normalize_input(user_input)
        if difflib.SequenceMatcher(None, normalized_user_input, normalized_riddle_answer).ratio() > 0.85:
            current_mode = None
            return "Well done! You've solved the riddle! Type another spell to continue."
        else:
            current_mode = None
            return f"Not quite! The answer was '{current_riddle_answer}'. Would you like to try another riddle or cast a spell?"

    # Check for commands
    intent = get_intent(user_input_normalized)
    if intent:
        if intent == 'fetch_trivia':
            current_question, current_answer = handle_trivia()
            current_mode = 'trivia'
            return f"Here's a trivia question for you: {current_question}"
        elif intent == 'fetch_weather':
            return "The weather today is sunny with a hint of magic in the air!"
        elif intent == 'fetch_riddles':
            current_riddle, current_riddle_answer = handle_riddles()
            current_mode = 'riddle'
            return f"Here's a riddle for you: {current_riddle}"
        elif intent == 'give_advice':
            return "Believe in yourself! You have the power to create your own magic."
        elif intent == 'end_conversation':
            return "Farewell, brave spellcaster! Until we meet again."
        elif intent == 'fetch_fun_fact':
            return "Did you know? The first Harry Potter book was published in 1997!"
        elif intent == 'fetch_joke':
            return "Why did the wizard break up with his girlfriend? Because she had too many hexes!"
        elif intent == 'fetch_prophecy':
            return "Today is a day of great potential. Use your magic wisely!"
        elif intent == 'fetch_knowledge':
            return handle_knowledge()
        elif intent == 'fetch_history':
            return handle_history()
        elif intent == 'fetch_explanation':
            return handle_explanation()
        elif intent == 'fetch_question':
            return handle_question()
        elif intent == 'cheer_user':
            return handle_cheer()
        elif intent == 'fetch_details':
            return handle_details()
        elif intent == 'fetch_movie':
            return handle_movie()

    return "Sorry, I don't understand that spell. Can you try again?"

In [74]:
# --- Step 6: GUI Setup ---
root = Tk()
root.title("SpellBinder")
root.geometry("500x500")  # Smaller window size
root.configure(bg="#4B0082")

font = Font(family="Arial", size=12, weight="bold")


In [75]:
# Text area size adjustment
text_area = scrolledtext.ScrolledText(root, wrap=WORD, width=50, height=10, font=font, bg="#f0f8ff", fg="#2e0854", borderwidth=2, relief="solid")
text_area.pack(padx=10, pady=10)

entry = Entry(root, width=50, font=font, bg="#e6e6fa", fg="#2e0854", borderwidth=2)
entry.pack(pady=5)

In [76]:
def on_send():
    user_input = entry.get()
    if user_input.strip():
        response = get_response(user_input)
        text_area.insert(END, f"You: {user_input}\nSpellBinder: {response}\n\n")  # Updated assistant name to SpellBinder
        entry.delete(0, END)
        text_area.yview(END)

send_button = Button(root, text="Send", font=font, command=on_send, bg="#8a2be2", fg="white", relief="flat")
send_button.pack(pady=5)

In [77]:
# Spell display area
spells_frame = Frame(root, bg="#4B0082")
spells_frame.pack(pady=10)

spell_list = ["Lumos", "Alohomora", "Expelliarmus", "Wingardium Leviosa", "Accio", "Expecto Patronum"]

for spell in spell_list:
    spell_button = Button(spells_frame, text=spell, font=font, bg="#8a2be2", fg="white", relief="flat")
    spell_button.pack(side=LEFT, padx=5)

In [78]:
root.mainloop()